In [3]:
from http.cookiejar import domain_match

from pydantic import BaseModel, create_model
import json

# Data validation against schema
from jsonschema import validate
from jsonschema.exceptions import ValidationError

In [ ]:
#### Azure AD Data
azure_ad_data = None
with open("../data/caesars/2024-11-20_azure_ad.json", 'r') as f:
    azure_ad_data = json.load(f)

In [4]:
okta_data = None
# Okta data
with open("../data/caesars/okta/2024-11-20_okta_logs_1.json", 'r') as f:
    okta_data = json.load(f)

In [12]:
# zone
# domain
# # device os and other details to join on device
n=0
node = dict()
for i in okta_data:
    if i.get("device"):
        if len(i.get("device").keys()) > n:
            n = len(i.get("device").keys())
            node = i.get("device")

In [7]:
okta_data[:2]

[{'actor': {'id': '00uiqmzbfeutBIKmD1t7',
   'type': 'User',
   'alternateId': 'Teresa.Drace@islebettendorf.com',
   'displayName': 'Teresa Drace',
   'detailEntry': None},
  'client': {'userAgent': {'rawUserAgent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'os': 'Windows 10',
    'browser': 'CHROME'},
   'zone': 'CaesarsNetworkCurrent',
   'device': 'Computer',
   'id': None,
   'ipAddress': '216.221.27.101',
   'geographicalContext': {'city': 'Saint Paul',
    'state': 'Minnesota',
    'country': 'United States',
    'postalCode': '55128',
    'geolocation': {'lat': 44.9926, 'lon': -92.9537}}},
  'device': None,
  'authenticationContext': {'authenticationProvider': None,
   'credentialProvider': None,
   'credentialType': None,
   'issuer': None,
   'interface': None,
   'authenticationStep': 0,
   'rootSessionId': 'idxECowc1fgQ2uQIbc8L9sVYw',
   'externalSessionId': 'idxECowc1fgQ2uQIbc8L9sVYw'},
  'displayM

In [13]:
node

{'id': 'guo12h435gtFbQ6kP1t8',
 'name': 'Mac15,7',
 'os_platform': 'OSX',
 'os_version': '14.1.0',
 'managed': False,
 'registered': True,
 'device_integrator': '{"WSC":{},"CALLER_CONTEXT":{},"CROWDSTRIKE":{}}',
 'disk_encryption_type': 'ALL_INTERNAL_VOLUMES',
 'screen_lock_type': 'BIOMETRIC',
 'jailbreak': None,
 'secure_hardware_present': True}

In [ ]:
from typing import Any, Optional, List, Dict, get_origin, get_args
from pydantic import BaseModel, create_model

def infer_type(values: List[Any]):
    types = {type(v) for v in values if v is not None}
    nullable = any(v is None for v in values)

    if types.issubset({int}):
        base = int
    elif types.issubset({float, int}):
        base = float
    elif types.issubset({str}):
        base = str
    elif types and all(isinstance(v, dict) for v in values if v is not None):
        base = dict
    elif types and all(isinstance(v, list) for v in values if v is not None):
        base = list
    else:
        base = Any

    if nullable:
        return Optional[base]
    return base

def build_model_from_samples(name: str, samples: List[dict]) -> BaseModel:
    # gather all keys
    all_keys = set().union(*samples)
    fields = {}

    for key in all_keys:
        vals = [d.get(key) for d in samples]
        required = all(d.get(key) is not None for d in samples)

        # nested dict/list handling
        if all(isinstance(v, dict) or v is None for v in vals):
            sub_samples = [v for v in vals if isinstance(v, dict)]
            sub_model = build_model_from_samples(f"{name}_{key}", sub_samples) if sub_samples else Any
            typ = Optional[sub_model] if not required else sub_model
        elif all(isinstance(v, list) or v is None for v in vals):
            inner = [item for v in vals if isinstance(v, list) for item in v]
            elem_type = infer_type(inner) if inner else Any
            typ = List[elem_type]
            if not required:
                typ = Optional[typ]
        else:
            typ = infer_type(vals)

        default = ... if required else None
        fields[key] = (typ, default)

    return create_model(name, **fields)



In [ ]:
#for item in okta_data:
 #   validated = MyModel.model_validate(item)  # Pydantic v2
  #  print(validated)

In [ ]:
okta_model = build_model_from_samples("OktaSystemLogs", okta_data)
ad_model = build_model_from_samples("ActiveDirectoryAzureUsers", azure_ad_data['users'])



In [ ]:
ad_model.model_json_schema()

In [ ]:

def make_model_from_dict(name: str, sample: dict) -> BaseModel:
    fields = {}
    for k, v in sample.items():
        if isinstance(v, dict):
            sub = make_model_from_dict(f"{name}_{k}", v)
            fields[k] = (sub, ...)
        elif isinstance(v, list) and v and isinstance(v[0], dict):
            sub = make_model_from_dict(f"{name}_{k}", v[0])
            from typing import List
            fields[k] = (List[sub], ...)
        else:
            fields[k] = (type(v), ...)
    return create_model(name, **fields)

# Example usage with your data:
# item = {
#     "actor": {"id": "00uiqmz...", "multiFactorMode": "1FA"}
# }
DynamicModel = make_model_from_dict("DynamicModel", okta_data[0])
# instance = DynamicModel(**okta_data[0])

In [ ]:
DynamicModel.model_json_schema()

In [ ]:
okta_data[0]

In [ ]:
from pydantic import create_model, Field
from typing import List, Optional

def make_model_from_dict(name: str, sample: dict) -> BaseModel:
    def infer(k, v):
        if isinstance(v, dict):
            sub = make_model_from_dict(f"{name}_{k}", v)
            return sub, ...
        elif isinstance(v, list) and v and isinstance(v[0], dict):
            sub = make_model_from_dict(f"{name}_{k}", v[0])
            return List[sub], ...
        else:
            return type(v), ...

    fields = {k: infer(k, v) for k, v in sample.items()}
    return create_model(name, **fields)


In [ ]:
from datamodel_code_generator import InputFileType, generate
from pathlib import Path
from tempfile import TemporaryDirectory
from datamodel_code_generator import DataModelType
# o_path = "../data/test/"
# Load your JSON or dict
json_string = json.dumps(azure_ad_data)
# sample = Path("../data/caesars/okta/2024-11-20_okta_logs_1.json").read_text()
# output = Path(o_path / "model.py")
with TemporaryDirectory() as temporary_directory_name:
    temporary_directory = Path(temporary_directory_name)
    output = Path(temporary_directory / "model.py")
    generate(
        json_string,
        input_file_type=InputFileType.Json,
        input_filename="2024-11-20_azure_ad.json",
        output=output,
        # set up the output model types
        output_model_type=DataModelType.PydanticV2BaseModel,
    )
    model: str = output.read_text()
print(model)

#### Generate JMES Paths for all the keys
Create paths to each key name to account for duplicate key name values at different level in nested objects.
jmes_path = JMES Path
context = string describing the key value pairs meaning in context to the data


In [ ]:
import jmespath
from collections.abc import Mapping, Sequence

def add_jmes_paths(dicts: list[dict], root_name: str = "root") -> None:
    """
    Recursively adds "*_jmes_path" fields for each primitive leaf in nested dicts and lists.
    Operates in-place; iterates over a copy of keys to avoid mutation issues.
    """
    def recurse(data, prefix):
        if isinstance(data, Mapping):
            for key, value in list(data.items()):
                if key.endswith("_jmes_path"):
                    continue
                new_prefix = f"{prefix}.{key}"
                recurse(value, new_prefix)
        elif isinstance(data, Sequence) and not isinstance(data, (str, bytes)):
            for idx, item in enumerate(data):
                recurse(item, f"{prefix}[{idx}]")
        else:
            # Leaf node: validate and assign path
            try:
                jmespath.compile(prefix)
            except jmespath.exceptions.JMESPathError as exc:
                raise ValueError(f"Invalid JMESPath '{prefix}': {exc}") from exc
            return False  # handled by parent caller
        return True

    for data in dicts:
        recurse(data, root_name)


In [ ]:
def collect_jmes_paths(obj, prefix=""):
    paths = []
    if isinstance(obj, dict):
        for key, val in obj.items():
            jkey = f"{prefix}.{key}" if prefix else key
            paths.append(jkey)
            paths.extend(collect_jmes_paths(val, jkey))
    elif isinstance(obj, list):
        # Use [*] to denote any index
        jkey = f"{prefix}[*]"
        paths.append(jkey)
        for item in obj:
            paths.extend(collect_jmes_paths(item, jkey))
    return paths


In [ ]:
import copy
import jmespath
from collections.abc import Mapping, Sequence
from typing import Any, Dict, List, Tuple, Union
from traceback import format_exc
JSON = Union[Dict[str, Any], List[Any]]

def annotate_with_jmes_paths(
    data: JSON
) -> Tuple[JSON, List[str]]:
    """
    Deepcopy the input (dict or list of dicts), then:
      - Traverse nested dicts/lists recursively.
      - For each dict encountered, collect JMESPaths for all its primitive leaves.
      - Add 'jmes_paths' key with that list.
    Returns (annotated_clone, flat_list_of_paths).
    """
    cloned = copy.deepcopy(data)
    all_paths: List[str] = []

    def recurse(obj: Any, path_parts: List[str]) -> List[str]:
        """
        Returns list of JMESPaths for primitive leaves under this object.
        Also updates nested dicts to include 'jmes_paths'.
        """
        paths_here: List[str] = []

        if isinstance(obj, Mapping):
            for key, value in obj.items():
                child_parts = path_parts + [key]
                # Recursively gather for nested structures
                if isinstance(value, Mapping) or (isinstance(value, Sequence) and not isinstance(value, (str, bytes))):
                    child_paths = recurse(value, child_parts)
                    paths_here.extend(child_paths)
                else:
                    # Primitive leaf: build path, validate, record
                    expr = ".".join(child_parts).replace(".[", "[")
                    try:
                        jmespath.compile(expr)
                    except jmespath.exceptions.JMESPathError as exc:
                        raise ValueError(f"Invalid JMESPath '{expr}': {exc}") from exc
                    paths_here.append(expr)
            if paths_here:
                obj["jmes_paths"] = paths_here.copy()
        elif isinstance(obj, Sequence) and not isinstance(obj, (str, bytes)):
            for idx, item in enumerate(obj):
                child_paths = recurse(item, path_parts + [f"[{idx}]"])
                paths_here.extend(child_paths)

        return paths_here

    if isinstance(cloned, list):
        for item in cloned:
            all_paths.extend(recurse(item, []))
    else:
        all_paths.extend(recurse(cloned, []))

    return cloned, all_paths

def validate_paths_work(
    original: JSON, annotated: JSON
) -> Tuple[bool, List[Tuple[str, Any, Any]]]:
    """
    Checks each path in 'jmes_paths' of each dict in `annotated`.
    Confirms that jmespath.search(path, original_data) == fetched value.
    Returns (all_ok: bool, failures: List[(path, expected, actual)]).
    """
    failures: List[Tuple[str, Any, Any]] = []

    def recurse(obj: Any):
        if isinstance(obj, Mapping):
            current_keys = obj.keys()
            for key, value in obj.items():
                if key == "jmes_paths":
                    for path in value:
                        field_name = path.split(".")[-1]
                        try:
                            # expected = jmespath.search(path, original)
                            actual = jmespath.search(path, annotated)
                            print(actual)
                            print(obj[field_name])
                            if obj.get(field_name):
                                if actual != obj.get(field_name):
                                    if actual is not None:
                                        failures.append((path, obj[field_name], actual))
                            # if expected != actual:
                            #     failures.append((path, expected, actual))
                        except KeyError:
                            print(f"key not found, skipping: {path} \n{format_exc()}")
                            print(obj)
                            continue
                        except jmespath.exceptions.JMESPathError as exc:
                            failures.append((path,obj.get(field_name), None))
                            print(f"Invalid JMESPath '{path}': {exc}")
                            continue
                        except Exception as e:
                            failures.append((path,obj.get(field_name), None))
                            print(f"Error: '{path}': {e}\n{format_exc()}")
                else:
                    recurse(value)
        elif isinstance(obj, Sequence) and not isinstance(obj, (str, bytes)):
            for item in obj:
                recurse(item)

    recurse(annotated)
    return (len(failures) == 0, failures)


In [ ]:
okta_data_jmes = annotate_with_jmes_paths(okta_data[:20])

In [ ]:
o_data, paths = okta_data_jmes


In [ ]:
success_bool, fail_report = validate_paths_work(okta_data[:20], o_data)

In [ ]:
fail_report[0]

#### Okta Data Models
These are the extracted data models from the Okta system logs data from caesars.

In [ ]:
# generated by datamodel-codegen:
#   filename:  2024-11-20_okta_logs_1.json
#   timestamp: 2025-06-16T14:12:39+00:00

from __future__ import annotations

from typing import List, Optional

from pydantic import BaseModel, RootModel

class JMESPaths(BaseModel):
    jmes_path: list[str]

class Actor(BaseModel):
    id: str
    type: str
    alternateId: str
    displayName: str
    detailEntry: None
    jmes_path: JMESPaths


class UserAgent(BaseModel):
    rawUserAgent: str
    os: str
    browser: str
    jmes_path: JMESPaths

class Geolocation(BaseModel):
    lat: float
    lon: float
    jmes_path: JMESPaths

class GeographicalContext(BaseModel):
    city: Optional[str] = None
    state: Optional[str] = None
    country: str
    postalCode: Optional[str] = None
    geolocation: Geolocation
    jmes_path: JMESPaths


class Client(BaseModel):
    userAgent: UserAgent
    zone: str
    device: str
    id: Optional[str] = None
    ipAddress: str
    geographicalContext: GeographicalContext
    jmes_path: JMESPaths

class Device(BaseModel):
    id: str
    name: str
    os_platform: str
    os_version: str
    managed: bool
    registered: bool
    device_integrator: Optional[str] = None
    disk_encryption_type: str
    screen_lock_type: str
    jailbreak: Optional[bool] = None
    secure_hardware_present: bool
    jmes_path: JMESPaths

class AuthenticationContext(BaseModel):
    authenticationProvider: Optional[str] = None
    credentialProvider: Optional[str] = None
    credentialType: Optional[str] = None
    issuer: None
    interface: None
    authenticationStep: int
    rootSessionId: str
    externalSessionId: str
    jmes_path: JMESPaths

class Outcome(BaseModel):
    result: str
    reason: Optional[str] = None
    jmes_path: JMESPaths

class SecurityContext(BaseModel):
    asNumber: Optional[int] = None
    asOrg: Optional[str] = None
    isp: Optional[str] = None
    domain: Optional[str] = None
    isProxy: bool
    jmes_path: JMESPaths

class DebugData(BaseModel):
    authnRequestId: Optional[str] = None
    requestId: str
    authMethodFirstVerificationTime: Optional[str] = None
    dtHash: Optional[str] = None
    authMethodFirstType: Optional[str] = None
    authMethodFirstEnrollment: Optional[str] = None
    requestUri: str
    threatSuspected: Optional[str] = None
    url: str
    logOnlySecurityData: Optional[str] = None
    authMethodSecondVerificationTime: Optional[str] = None
    authMethodSecondEnrollment: Optional[str] = None
    authMethodSecondType: Optional[str] = None
    redirectUri: Optional[str] = None
    behaviors: Optional[str] = None
    challengeAuthenticatorsList: Optional[str] = None
    risk: Optional[str] = None
    oktaUserAgentExtended: Optional[str] = None
    deviceFingerprint: Optional[str] = None
    origin: Optional[str] = None
    loginResult: Optional[str] = None
    traceId: Optional[str] = None
    factor: Optional[str] = None
    factorIntent: Optional[str] = None
    authenticatorMethodChallengeTime: Optional[str] = None
    userStatus: Optional[str] = None
    tunnels: Optional[str] = None
    pushOnlyResponseType: Optional[str] = None
    pushWithNumberChallengeResponseType: Optional[str] = None
    authMethodThirdEnrollment: Optional[str] = None
    authMethodThirdType: Optional[str] = None
    authMethodThirdVerificationTime: Optional[str] = None
    threatDetections: Optional[str] = None
    authMethodFourthVerificationTime: Optional[str] = None
    authMethodFourthType: Optional[str] = None
    authMethodFourthEnrollment: Optional[str] = None
    keyTypeUsedForAuthentication: Optional[str] = None
    invalidToken: Optional[str] = None
    jmes_path: JMESPaths

class DebugContext(BaseModel):
    debugData: DebugData
    jmes_path: JMESPaths

class Detail(BaseModel):
    rootApiTokenId: Optional[str] = None
    requestApiTokenId: Optional[str] = None
    requestApiTokenClientId: Optional[str] = None
    jmes_path: JMESPaths

class Transaction(BaseModel):
    type: str
    id: str
    detail: Detail
    jmes_path: JMESPaths

class IpChainItem(BaseModel):
    ip: str
    geographicalContext: GeographicalContext
    version: str
    source: None
    jmes_path: JMESPaths

class Request(BaseModel):
    ipChain: List[IpChainItem]
    jmes_path: JMESPaths

class DetailEntry(BaseModel):
    signOnModeType: Optional[str] = None
    signOnModeEvaluationResult: Optional[str] = None
    policyRuleFactorMode: Optional[str] = None
    deviceIntegrator: Optional[str] = None
    osVersion: Optional[str] = None
    managed: Optional[str] = None
    osPlatform: Optional[str] = None
    tpmPresent: Optional[str] = None
    deviceStatus: Optional[str] = None
    methodTypeUsed: Optional[str] = None
    policyType: Optional[str] = None
    jmes_path: JMESPaths

class TargetItem(BaseModel):
    id: str
    type: str
    alternateId: str
    displayName: str
    detailEntry: Optional[DetailEntry] = None
    jmes_path: JMESPaths

class OktaItemModel(BaseModel):
    actor: Actor
    client: Client
    device: Optional[Device] = None
    authenticationContext: AuthenticationContext
    displayMessage: str
    eventType: str
    outcome: Outcome
    published: str
    securityContext: SecurityContext
    severity: str
    debugContext: DebugContext
    legacyEventType: Optional[str] = None
    transaction: Transaction
    uuid: str
    version: str
    request: Request
    target: Optional[List[TargetItem]] = None
    jmes_path: JMESPaths

class OktaSysLogModel(RootModel[List[OktaItemModel]]):
    root: List[OktaItemModel]



In [ ]:
okta_data_model = OktaSysLogModel(okta_data)

In [ ]:
okta_data_model.model_json_schema()

### Azure AD Data Models
Data model for Azure AD data categories:
- Users
-

In [ ]:
# generated by datamodel-codegen:
#   filename:  2024-11-20_azure_ad.json
#   timestamp: 2025-06-16T14:15:16+00:00

from __future__ import annotations

from typing import List, Optional

from pydantic import BaseModel, Field


class Device(BaseModel):
    id: str
    displayName: str
    operatingSystem: str
    operatingSystemVersion: Optional[str] = None
    profileType: Optional[str] = None
    accountEnabled: bool
    approximateLastSignInDateTime: Optional[str] = None
    createdDateTime: str
    deviceId: str
    isManaged: Optional[bool] = None
    onPremisesSyncEnabled: Optional[bool] = None
    registrationDateTime: Optional[str] = None
    trustType: Optional[str] = None
    deviceVersion: Optional[int] = None
    deviceOwnership: Optional[str] = None
    enrollmentType: Optional[str] = None
    isCompliant: Optional[bool] = None
    isRooted: Optional[bool] = None
    managementType: Optional[str] = None
    manufacturer: Optional[str] = None
    mdmAppId: Optional[str] = None
    model: Optional[str] = None
    hostnames: Optional[List[str]] = None
    systemLabels: Optional[List[str]] = None
    enrollmentProfileName: Optional[str] = None

class DeviceEndpoint(BaseModel):
    displayName: str
    operatingSystem: Optional[str] = None
    operatingSystemVersion: Optional[str] = None
    accountEnabled: bool
    approximateLastSignInDateTime: Optional[str] = None
    createdDateTime: str
    deviceId: str
    deviceVersion: Optional[int] = None
    registrationDateTime: Optional[str] = None
    trustType: Optional[str] = None
    id: str
    registeredOwners_delta: Optional[List[RegisteredOwnersDeltaItem]] = Field(
        None, alias='registeredOwners@delta'
    )
    registeredUsers_delta: Optional[List[RegisteredUsersDeltaItem]] = Field(
        None, alias='registeredUsers@delta'
    )
    profileType: Optional[str] = None
    onPremisesSyncEnabled: Optional[bool] = None
    hostnames: Optional[List[str]] = None
    isCompliant: Optional[bool] = None
    deviceOwnership: Optional[str] = None
    enrollmentType: Optional[str] = None
    isManaged: Optional[bool] = None
    isRooted: Optional[bool] = None
    managementType: Optional[str] = None
    manufacturer: Optional[str] = None
    mdmAppId: Optional[str] = None
    model: Optional[str] = None
    systemLabels: Optional[List[str]] = None
    enrollmentProfileName: Optional[str] = None


class ManagerDeltaItem(BaseModel):
    field_odata_type: str = Field(..., alias='@odata.type')
    id: str


class Identity(BaseModel):
    signInType: str
    issuer: str
    issuerAssignedId: Optional[str] = None


class User(BaseModel):
    displayName: Optional[str] = None
    mailNickname: Optional[str] = None
    givenName: Optional[str] = None
    surname: Optional[str] = None
    mail: Optional[str] = None
    proxyAddresses: Optional[List[str]] = None
    userType: Optional[str] = None
    accountEnabled: Optional[bool] = None
    onPremisesDistinguishedName: Optional[str] = None
    userPrincipalName: Optional[str] = None
    onPremisesSamAccountName: Optional[str] = None
    id: str
    transitiveMemberOf: List[str]
    devices: List[Device]
    jobTitle: Optional[str] = None
    department: Optional[str] = None
    companyName: Optional[str] = None
    manager_delta: Optional[List[ManagerDeltaItem]] = Field(None, alias='manager@delta')
    manager: Optional[str] = None
    passwordPolicies: Optional[str] = None
    city: Optional[str] = None
    state: Optional[str] = None
    streetAddress: Optional[str] = None
    country: Optional[str] = None
    otherMails: Optional[List[str]] = None
    identities: Optional[List[Identity]] = None
    employeeType: Optional[str] = None
    deletedDateTime: None = None


class MembersDeltaItem(BaseModel):
    field_odata_type: str = Field(..., alias='@odata.type')
    id: str


class Group(BaseModel):
    createdDateTime: Optional[str] = None
    displayName: Optional[str] = None
    mailEnabled: Optional[bool] = None
    mailNickname: Optional[str] = None
    onPremisesDomainName: Optional[str] = None
    onPremisesSecurityIdentifier: Optional[str] = None
    onPremisesSyncEnabled: Optional[bool] = None
    renewedDateTime: Optional[str] = None
    securityEnabled: Optional[bool] = None
    securityIdentifier: Optional[str] = None
    id: str
    description: Optional[str] = None
    members_delta: Optional[List[MembersDeltaItem]] = Field(None, alias='members@delta')
    mail: Optional[str] = None
    proxyAddresses: Optional[List[str]] = None
    groupTypes: Optional[List[str]] = None
    deletedDateTime: None = None
    createdByAppId: Optional[str] = None


class RegisteredOwnersDeltaItem(BaseModel):
    field_odata_type: str = Field(..., alias='@odata.type')
    id: str


class RegisteredUsersDeltaItem(BaseModel):
    field_odata_type: str = Field(..., alias='@odata.type')
    id: str



class DeltaUrls(BaseModel):
    users: str
    groups: str
    devices: str


class Model(BaseModel):
    users: List[User]
    groups: List[Group]
    devices: List[DeviceEndpoint]
    delta_urls: DeltaUrls

